# Sinusoidal trajectory generator

Import libraries:

In [78]:
# Imports required for data processing
import os

# Imports required for dynamics calculations
import math
from math import pi
import numpy as np
import matplotlib.pyplot as plt
import sympy as sym


# Imports required for animation
from plotly.offline import init_notebook_mode, iplot
from IPython.display import display, HTML
import plotly.graph_objects as go


Parameters:

In [79]:
# Masses, length and center-of-mass positions (calculated using the lab measurements)
# Mass calculations (mass unit is kg) 
m_body = 90.6                            # Average weights for American adult male
                                            # from "Anthropometric Reference Data for Children and Adults:
                                            # United States, 2015–2018"
m_body_dict = {'ID': 51.0, 'JD': 79.5, 'JR': 76.0, 'KS': 59.3, 'KW': 63.8, 'LC': 61.2,
                'LD': 97.3, 'LS': 82.2, 'MK': 93.5, 'MV': 98.5, 'SM': 68.5, 'TD': 70.0,
                'TM': 66.2}

m_u = 0.028 * m_body                     # Average upper arm weights relative to body weight, from “Biomechanics
                                            # and Motor Control of Human Movement” by David Winter (2009), 4th edition
m_u_dict = {'ID': 0.028 * m_body_dict['ID'], 'JD': 0.028 * m_body_dict['JD'],
            'JR': 0.028 * m_body_dict['JR'], 'KS': 0.028 * m_body_dict['KS'],
            'KW': 0.028 * m_body_dict['KW'], 'LC': 0.028 * m_body_dict['LC'],
            'LD': 0.028 * m_body_dict['LD'], 'LS': 0.028 * m_body_dict['LS'],
            'MK': 0.028 * m_body_dict['MK'], 'MV': 0.028 * m_body_dict['MV'],
            'SM': 0.028 * m_body_dict['SM'], 'TD': 0.028 * m_body_dict['TD'],
            'TM': 0.028 * m_body_dict['TM']}

m_l = 0.7395                             # Average lower prosthetics weights, calculated using lab measurements  

# Arm length calculations (length unit is m) 
H_body = 1.769                           # Average height for American adult male, from “Height and body-mass 
                                            # index trajectories of school-aged children and adolescents from 
                                            # 1985 to 2019 in 200 countries and territories: a pooled analysis 
                                            # of 2181 population-based studies with 65 million participants”
H_body_dict = {'ID': 1.620, 'JD': 1.760, 'JR': 1.770, 'KS': 1.640, 'KW': 1.620, 'LC': 1.580,
                'LD': 1.875, 'LS': 1.635, 'MK': 1.780, 'MV': 1.805, 'SM': 1.790, 'TD': 1.690,
                'TM': 1.735}

L_u = 0.186 * H_body                     # Average upper arm length relative to body height
                                            # from “Biomechanics and Motor Control of Human Movement” by David
                                            # Winter (2009), 4th edition
L_u_dict = {'ID': 0.186 * H_body_dict['ID'], 'JD': 0.186 * H_body_dict['JD'],
            'JR': 0.186 * H_body_dict['JR'], 'KS': 0.186 * H_body_dict['KS'],
            'KW': 0.186 * H_body_dict['KW'], 'LC': 0.186 * H_body_dict['LC'],
            'LD': 0.186 * H_body_dict['LD'], 'LS': 0.186 * H_body_dict['LS'],
            'MK': 0.186 * H_body_dict['MK'], 'MV': 0.186 * H_body_dict['MV'],
            'SM': 0.186 * H_body_dict['SM'], 'TD': 0.186 * H_body_dict['TD'],
            'TM': 0.186 * H_body_dict['TM']}

L_l = 0.42                               # Average lower prosthetics length, calculated using lab measurements

# Arm center of mass length calculations (length unit is m) 
L_u_c = 0.436 * L_u                      # Average upper arm length from shoulder to center of mass relative
                                            # to upper arm length, from “Biomechanics and Motor Control of Human
                                            # Movement” by David Winter (2009), 4th edition
L_u_c_dict = {'ID': 0.436 * L_u_dict['ID'], 'JD': 0.436 * L_u_dict['JD'],
                'JR': 0.436 * L_u_dict['JR'], 'KS': 0.436 * L_u_dict['KS'],
                'KW': 0.436 * L_u_dict['KW'], 'LC': 0.436 * L_u_dict['LC'],
                'LD': 0.436 * L_u_dict['LD'], 'LS': 0.436 * L_u_dict['LS'],
                'MK': 0.436 * L_u_dict['MK'], 'MV': 0.436 * L_u_dict['MV'],
                'SM': 0.436 * L_u_dict['SM'], 'TD': 0.436 * L_u_dict['TD'],
                'TM': 0.436 * L_u_dict['TM']}
L_l_c = 0.2388                           # Average lower prosthetics length from elbow to center of mass,
                                            # calculated using lab measurements 

In [84]:

# Does not work with run all, will be cut off short while generating trajectory
# Parameters
size = 3998
frame_rate = 100

freq = [1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5 ,5.5 ,6 ,6.5 ,7 ,7.5, 8, 8.5]
amp = [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]
print(len(freq),len(amp))
for i in range(len(freq)):
    # Define variables
    t = sym.Symbol(r't')

    # Define the function
    a = amp[i]
    fr = freq[i]
    f = a*sym.sin(t*fr)

    # And its dervatives
    df = sym.diff(f)
    ddf = sym.diff(df)

    # Lambdify
    lam1 = sym.lambdify('t', f)
    lam2 = sym.lambdify('t', df)
    lam3 = sym.lambdify('t', ddf)

    # Open file
    Angle = open(f"CSV_Converted_Files/{freq[i]:02f}_{amp[i]}_angle.csv", 'w')
    Vel = open(f"CSV_Converted_Files/{freq[i]:02f}_{amp[i]}_velocity.csv",'w')
    Acc = open(f"CSV_Converted_Files/{freq[i]:02f}_{amp[i]}_accel.csv",'w')

    # Create the trajectory and write it to the file
    period = math.ceil(((2 * pi * frame_rate )/ freq[i]) + 1)

    print(f'#define DATA_SAMPLES_{int(freq[i] * 10)}  {period}')
    for ii in range(period):
        
        lines = [str(lam1(ii/frame_rate)) + '\n', str(lam2(ii/frame_rate)) + '\n', str(lam3(ii/frame_rate)) + '\n']
        Angle.write(lines[0])
        Vel.write(lines[1])
        Acc.write(lines[2])

    # Close the files to ensure all data is flushed
    Angle.close()
    Vel.close()
    Acc.close()

16 16
#define DATA_SAMPLES_10  630
#define DATA_SAMPLES_15  420
#define DATA_SAMPLES_20  316
#define DATA_SAMPLES_25  253
#define DATA_SAMPLES_30  211
#define DATA_SAMPLES_35  181
#define DATA_SAMPLES_40  159
#define DATA_SAMPLES_45  141
#define DATA_SAMPLES_50  127
#define DATA_SAMPLES_55  116
#define DATA_SAMPLES_60  106
#define DATA_SAMPLES_65  98
#define DATA_SAMPLES_70  91
#define DATA_SAMPLES_75  85
#define DATA_SAMPLES_80  80
#define DATA_SAMPLES_85  75


In [85]:
data_csv_dir = 'CSV_Converted_Files'
print("current directory: ", os.getcwd())

frame_frequency = frame_rate


walking_vel_list = []


file_list = os.listdir(data_csv_dir)
file_list.sort()
file_list_temp = file_list

print(file_list)

# save the torque and acceleration profile and plot
with open("src/data_profile_sinusoid_array.c", "w") as data_file:
    data_file.write('#include "data_profile.h"\n\n')
    for i in range(len(freq)):
        time_list = []
        elbow_ang_list = []
        elbow_vel_list = []
        elbow_acc_list = []
        elbow_acc_data_list = []

        theta_list = []
        dtheta_list = []
        ddtheta_list = []

        file_list = file_list_temp
        for file in file_list:

            file_name = file[:-4]


            frame = 0
            file_time_list = []
            file_elbow_ang_list, file_elbow_vel_list, file_elbow_acc_list, time_list = [], [], [], []

            data_path = os.path.join(data_csv_dir, file)
            data_rows = open(data_path).read().strip().split("\n")
            
            # Record data
            if file_name == f"{freq[i]:02f}_{amp[i]}_angle":
                # print(file_name)
                for row in data_rows:
                    splitted_row = row.strip()
                    file_elbow_ang_list = splitted_row
                    elbow_ang_list.append(float(file_elbow_ang_list))
                    time_list.append(frame/frame_frequency)
                    frame += 1

            elif file_name == f"{freq[i]:02f}_{amp[i]}_velocity":
                # print(file_name)
                for row in data_rows:
                    splitted_row = row.strip()
                    file_elbow_vel_list = splitted_row
                    elbow_vel_list.append(float(file_elbow_vel_list))
                    time_list.append(frame/frame_frequency)
                    frame += 1
     

            elif file_name == f"{freq[i]:02f}_{amp[i]}_accel":
                # print(file_name) 
                for row in data_rows:
                    splitted_row = row.strip()
                    file_elbow_acc_list = splitted_row
                    elbow_acc_list.append(float(file_elbow_acc_list))
                    time_list.append(frame/frame_frequency)
                    frame += 1
                    
            if (len(elbow_ang_list) != 0 and len(elbow_vel_list) != 0 and len(elbow_acc_list) != 0):
                t_list = time_list
                theta_list = elbow_ang_list
                dtheta_list = elbow_vel_list
                ddtheta_list = elbow_acc_list

                # Compute the trajectory of the arm's motion
                N = int((max(t_list))/(1/frame_frequency))

                tvec = np.linspace(min(t_list), max(t_list), N)
                traj = np.zeros((3, N))

                for jj in range(N):
                    traj[0, jj] = elbow_ang_list[jj]
                    traj[1, jj] = elbow_vel_list[jj]
                    traj[2, jj] = elbow_acc_list[jj]

                data_file.write('float Angle_Profile_SIN_' + str(int(freq[i] * 10)) + '_' + str(int(amp[i]*10)) + '_[DATA_SAMPLES_' + str(int(freq[i] * 10)) + '] = {\n')
                list_str = ', \n'.join(map(str, theta_list))  
                data_file.write(list_str)
                data_file.write('\n};\n')
                
                data_file.write('float Accel_Profile_SIN_' + str(int(freq[i]* 10)) + '_' + str(int(amp[i]*10)) + '_[DATA_SAMPLES_' + str(int(freq[i] * 10)) + '-2] = {\n')
                accel_list_str = ', \n'.join(map(str, ddtheta_list[:-2])) 
                data_file.write(accel_list_str)
                data_file.write('\n};\n')

                print('extern float Angle_Profile_SIN_' + str(int(freq[i] * 10)) + '_' + str(int(amp[i]*10)) + '_[DATA_SAMPLES_' + str(int(freq[i] * 10)) + '];')
                print('extern float Accel_Profile_SIN_' + str(int(freq[i]* 10)) + '_' + str(int(amp[i]*10)) + '_[DATA_SAMPLES_' + str(int(freq[i] * 10)) + '-2];')

                # Calculate the length difference between the time list and the trajectory lists
                diff = (len(t_list) - len(traj[0]))

                # plt.figure(figsize=(15,5))
                # plt.suptitle('Angles Vs. Time', fontsize=20)
                # plt.plot(t_list[:-diff], traj[0])
                # plt.ylabel('Angle [rad]')
                # plt.xlabel('Time [sec]')
                # plt.xlim([0, math.ceil(max(tvec))])
                # plt.grid()
                # plt.title('Elbow Angle')
                # plt.show()

                # plt.figure(figsize=(15,5))
                # plt.suptitle('Angular Velocity Vs. Time', fontsize=20)
                # plt.plot(t_list[:-diff], traj[1])
                # plt.ylabel('Velocity [rad/sec]')
                # plt.xlabel('Time [sec]')
                # plt.xlim([0, math.ceil(max(tvec))])
                # plt.grid()
                # plt.title('Elbow Angular Velocity')
                # plt.show()

                # plt.figure(figsize=(15,5))
                # plt.suptitle('Angular Acceleration Vs. Time', fontsize=20)
                # plt.plot(t_list[:-diff], traj[2])
                # plt.ylabel('Acceleration [rad/sec^2]')
                # plt.xlabel('Time [sec]')
                # plt.xlim([0, math.ceil(max(tvec))])
                # plt.grid()
                # plt.title('Elbow Angular Acceleration')
                # plt.show()

                # Reset the lists
                theta_list = []
                dtheta_list = []
                ddtheta_list = []

                break


 


current directory:  /Users/michaeljenz/zephyrproject/zephyr/prosthetic_position_control
['1.000000_0.2_accel.csv', '1.000000_0.2_angle.csv', '1.000000_0.2_velocity.csv', '1.500000_0.2_accel.csv', '1.500000_0.2_angle.csv', '1.500000_0.2_velocity.csv', '2.000000_0.2_accel.csv', '2.000000_0.2_angle.csv', '2.000000_0.2_velocity.csv', '2.500000_0.2_accel.csv', '2.500000_0.2_angle.csv', '2.500000_0.2_velocity.csv', '3.000000_0.2_accel.csv', '3.000000_0.2_angle.csv', '3.000000_0.2_velocity.csv', '3.500000_0.2_accel.csv', '3.500000_0.2_angle.csv', '3.500000_0.2_velocity.csv', '4.000000_0.2_accel.csv', '4.000000_0.2_angle.csv', '4.000000_0.2_velocity.csv', '4.500000_0.2_accel.csv', '4.500000_0.2_angle.csv', '4.500000_0.2_velocity.csv', '5.000000_0.2_accel.csv', '5.000000_0.2_angle.csv', '5.000000_0.2_velocity.csv', '5.500000_0.2_accel.csv', '5.500000_0.2_angle.csv', '5.500000_0.2_velocity.csv', '6.000000_0.2_accel.csv', '6.000000_0.2_angle.csv', '6.000000_0.2_velocity.csv', '6.500000_0.2_accel.